# How does BPM correlate with song popularity over time? #
A computational essay by: Richard Li, Suki Sacks, Norah Evans 

# Introduction: #

Over the past few years of human existence, *everything* has gotten faster. Instant deliveries, messaging, food, etc. A common critique of the 21st century is that it's built on speed. Our group wondered if that holds true for songs. Have we begun enjoying faster songs as everything around us accelerates?

# Method #

In order to do this, we must accomplish 3 things:

1. Find a metric for popularity among songs in a year
2. Find the BPM of those songs
3. Average the BPM and chart the trends over the course of many years

Each of these things occur in 3 files:
1. billboard.py
2. bpm.py
3. visualization.py
